# Convert raw EEG data to BIDS

This script will take raw EEG data files (as saved by PyCorder or ASAlab) and convert them to BIDS-compatible files and folder structure, with all metadata.

You must save the raw files in the `source` subfolder of your study's `data` folder. Assumes you ran `init_BIDS_study.ipynb` to create your BIDS directory structure.

Be sure to modify the variables in the *Study Parameters* section below as appropriate for your study. Failing to set these properly will ensure systematic grief throughout your study... (but is fixable in batch by correcting the variables and re-running this script).

In [47]:
from os import path as op
import os
import json
# import configparser
import yaml
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper

## Study Parameters

Will import study-level parameters from `config.yml` in `bids_root`

In [87]:
# this shouldn't change if you run this script from its default location in code/import
bids_root = op.abspath('../..')

cfg_file = op.join(bids_root, 'config.yml')
with open(cfg_file, 'r') as f:
    config = yaml.load(f, Loader=Loader)

study_name = config['study_name']
task = config['task']
reb_info = config['reb_info']
authors = config['authors']
line_freq = int(config['line_freq']) 
instn = config['instn']
instn_addr = config['instn_addr']
instn_dept = config['instn_dept']
funding = config['funding'] 
license = config['license']
data_type = config['data_type']
amp_mfr = config['amp_mfr']
amp_model = config['amp_model']
sw_filt = config['sw_filt']
hw_filt = config['hw_filt']
eeg_ref = config['eeg_ref']
eeg_ground = config['eeg_ground']
cap_mfr = config['cap_mfr']
cap_model = config['cap_model']
eog = config['eog']
montage_fname = config['montage_fname']

## Change additional parameters according to your study

In [76]:
# list the names of input raw EEG files
in_subjs = ['BRYNNBCI_01', 'BRYNNBCI_02', 'BRYNNBCI_03']

# output data files will convert file names to generic sub-001, 002, ... names 
# _ASSUMES__ last two characters in in_subjs are the id number; prepends a leading 0
# BIDS will prepend 'sub-' to the numbers in out_subjs
out_subjs = [ '0' + s[-2:] for s in in_subjs]

# source_path is where the input source (raw) files live
source_path = op.join(bids_root, 'sourcedata', config['data_type'])

# raw_path is where the results of running this script will be saved
raw_path = op.join(bids_root, 'rawdata')


## Event details

Provide a dictionary mapping your stimuli to event code numbers in your raw EEG data.

This is only really necessary in this script if you want to do some fancy event code processing prior to standard epoching.

The dictionaries below are **just examples** and **should be replaced** based on the codes used in your study.

In [77]:
trial_id = {'Neutral/Inverted/Standard/L1':211, 'Neutral/Upright/Standard/L1':221, 
            'Neutral/Inverted/Standard/L2':212, 'Neutral/Upright/Standard/L2':222,
            'Neutral/Inverted/Standard/L3':213, 'Neutral/Upright/Standard/L3':223,
            'Neutral/Inverted/Standard/L4':214, 'Neutral/Upright/Standard/L4':224,
            'Neutral/Inverted/Standard/L5':215, 'Neutral/Upright/Standard/L5':225,
            'Neutral/Inverted/Standard/L6':216, 'Neutral/Upright/Standard/L6':226,

            'Green/Inverted/Standard/L1':111, 'Green/Upright/Standard/L1':121,
            'Green/Inverted/Standard/L2':112, 'Green/Upright/Standard/L2':122,
            'Green/Inverted/Standard/L3':113, 'Green/Upright/Standard/L3':123,
            'Green/Inverted/Standard/L4':114, 'Green/Upright/Standard/L4':124,
            'Green/Inverted/Standard/L5':115, 'Green/Upright/Standard/L5':125,
            'Green/Inverted/Standard/L6':116, 'Green/Upright/Standard/L6':126,
           }

target_highlight_id = {'Cue/L1':191, 'Cue/L2':192, 'Cue/L3':193, 'Cue/L4':194, 'Cue/L5':195, 'Cue/L6':196}


event_id = {**target_highlight_id, **trial_id} #, 'ignore':99999}

## Import necessary packages
In general you will not need to modify anything below this point.

In [78]:
import shutil
from glob import glob

import numpy as np 
# from matplotlib import pyplot as plt
import mne
# from mne.datasets import eegbci
mne.set_log_level('error')

from mne_bids import write_raw_bids, BIDSPath, update_sidecar_json
from mne_bids.stats import count_events

## Import BrainVision data, process events, & covert to BIDS

In [90]:

for subj_num, subject in enumerate(in_subjs):
    print(subject)
    # subject_id is for naming output files
    subject_id = out_subjs[subj_num]

    # --- Import raw files ---
    raw_fnames =  glob(source_path + '/' + subject + '*.vhdr')

    if len(raw_fnames) > 1:
        # if multiple input files, we need to concatenate them
        raw_list = []
        for f in raw_fnames:
            raw_list.append(mne.io.read_raw_brainvision(f, 
                                                        preload=False, 
                                                        eog=config['eog']).set_montage(montage_fname)
                           )
        raw = mne.concatenate_raws(raw_list)    

        # mne-bids doesn't handle concatenated raw files, unless we save them as .fif then reload:
        raw_tmp_fname = source_path + '/' + subject + '-raw.fif'
        raw.save(raw_tmp_fname, overwrite=True)
        raw = mne.io.read_raw(raw_tmp_fname)
        
    else:
        raw = mne.io.read_raw(raw_fnames.pop(), 
                              preload=False, 
                              eog=config['eog']).set_montage(montage_fname)
        
    # --- Event Processing ---
    # In here you could add fancy things like 
    #  recoding events, contingent events, etc.
    events, event_dict = mne.events_from_annotations(raw)
    event_dict = {**event_id, **event_dict}
    
    # --- Make BIDS ---
    bids_path = BIDSPath(subject=subject_id, 
                         task=config['task'], 
                         datatype=config['data_type'],
                         root=op.join(bids_root, raw_path))

    write_raw_bids(raw, bids_path, 
                   events_data=events, event_id=event_dict, 
                   overwrite=True)

    # Update metadata with additional info
    entries = { 'Manufacturer':config['amp_mfr'],
                'ManufacturersModelName':config['amp_model'],
                'PowerLineFrequency':config['line_freq'],
                'SoftwareFilters':config['sw_filt'],
                'HardwareFilters':config['hw_filt'],
                'EEGReference':config['eeg_ref'],
                'EEGGround':config['eeg_ground'],
                'CapManufacturer':config['cap_mfr'],
                'CapManufacturersModelName':config['cap_model'],
                'EthicsApprovals':config['reb_info'],
                'InstitutionName':config['instn'],
                'InstitutionAddress':config['instn_addr'],
                'InstitutionalDepartmentName':config['instn_dept'],
                }

    sidecar_path = bids_path.copy().update(extension='.json')
    update_sidecar_json(bids_path=sidecar_path, entries=entries)
    
    # Clean up
    if len(raw_fnames) > 1:
        os.remove(raw_tmp_fname)


BRYNNBCI_01
BRYNNBCI_02
BRYNNBCI_03
